In [164]:
client_id = ""
client_secret = ""
naver_id = ""
naver_pw = ""
import datetime
from pyspark import SparkContext
import glob
today = datetime.date.today()
content_file_name = "content_data/naver_news_content_today".replace("today",str(today))
title_file_name = "temp_data/naver_news_title_today".replace("today",str(today))
sc.stop()
sc = SparkContext(master="local", appName="spark app")

for i in range(1):
    print(str(i+1) + "번째 반복 중" )
    get_data(client_id, client_secret, naver_id, naver_pw, content_file_name, sc)
    get_model()
    get_topics()
    save_topics()

1번째 반복 중
start :: 식품 & 비타민 & 건강 & 푸드 & 제품 & 음식



end :: 식품 & 비타민 & 건강 & 푸드 & 제품 & 음식


In [160]:
def get_data(client_id, client_secret, naver_id, naver_pw, content_file_name, sc):
    import os
    import sys
    import urllib.request
    import requests

    import pandas as pd
    import numpy as np
    from selenium import webdriver
    from tqdm import notebook
    import pickle
    import re
    import ast

    from bs4 import BeautifulSoup
    import urllib
    import time

    news_data = []
    page_count = 10

    keywords_arr = sc.textFile(glob.glob("keyword/keyword*")[(len(glob.glob("keyword/keyword*"))-1)]).collect()
    keywords = ""
    for v in keywords_arr:
        keywords += v

    print("start :: " + keywords)
    encText = urllib.parse.quote(keywords)

    for idx in notebook.tqdm(range(page_count)):
        url = "https://openapi.naver.com/v1/search/news?query=" + encText + "&start=" + str(idx *10 +1)
        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id",client_id)
        request.add_header("X-Naver-Client-Secret",client_secret)
        response = urllib.request.urlopen(request)
        rescode = response.getcode()

        if(rescode==200):
            #response_body = response.read()
            result = requests.get(response.geturl(),
                             headers={"X-Naver-Client-Id":client_id,"X-Naver-Client-Secret":client_secret})
            news_data.append(result.json())
        else:
            print("Error Code:" + rescode)
        time.sleep(1)    


    naver_news_link = []

    for page in news_data:
        page_news_link=[]
        for item in page['items']:
            link = item['link']
            #page_news_link.append(link)
            if "naver" in link:
                page_news_link.append(link)
        naver_news_link.append(page_news_link)        

    link_count = 0        
    for page in naver_news_link:
         for link in page:
                link_count += 1 

    naver_news_title = []
    naver_news_content = []

    for n in notebook.tqdm(range(len(naver_news_link))):
        news_page_title = []
        news_page_content = []

        if n==0 or n%5 == 0:
            chrome_options = webdriver.ChromeOptions()
            chrome_options.add_argument('--no-sandbox')
            chrome_options.add_argument('--window-size=1420,1080')
            chrome_options.add_argument('--headless')
            chrome_options.add_argument('--disable-gpu')
            chrome_options.add_argument('--disable-dev-shm-usage')        
            driver = webdriver.Chrome(chrome_options=chrome_options)
            driver.get("https://nid.naver.com/nidlogin.login")
            driver.find_element_by_name('id').send_keys(naver_id)
            driver.find_element_by_name('pw').send_keys(naver_pw)

        for idx in notebook.tqdm(range(len(naver_news_link[n]))):
            time.sleep(2)

            try:

                driver.get(naver_news_link[n][idx])
            except:
                print("Timeout!")
                continue

            try:
                response = driver.page_source

            except UnexpectedAlertPresentException:
                driver.switch_to_alert().accept()
                print("게시글이 삭제됨")
                continue

            soup = BeautifulSoup(response, "html.parser")

            title = None

            try:
                item = soup.find('div', class_="article_info")
                title = item.find('h3', class_="tts_head").get_text()
            except:
                title = "OUTLINK"

            news_page_title.append(title)

            doc = None
            text= ""

            data = soup.find_all("div",{"class" : "_article_body_contents"})
            if data:
                for item in data:
                    text = text + str(item.find_all(text=True)).strip()
                    text = ast.literal_eval(text)
                    doc = ' '.join(text)
            else:
                doc = "OUTLINK"

            news_page_content.append(doc.replace('\n',' '))

        naver_news_title.append(news_page_title)
        naver_news_content.append(news_page_content)

    filecount=len(glob.glob("content_data/*"))
    sc.parallelize(naver_news_content).saveAsTextFile(content_file_name+"_"+str(filecount))

In [161]:
def get_model():
    from konlpy.tag import Mecab

    mecab = Mecab()
    import matplotlib.pyplot as plt
    %matplotlib inline
    # notebook.tqdm
    from konlpy.tag import Mecab
    import string
    import warnings
    from gensim import corpora
    from gensim import models

    warnings.filterwarnings("ignore",category=DeprecationWarning)
    # documents=[]
    # for filename in glob.glob("content_data/*"):
    #    documents += sc.textFile(filename).collect()
    documents = sc.textFile(glob.glob("content_data/*")[(len(glob.glob("content_data/*"))-1)]).collect()    
    SW = set()
    for i in string.punctuation:
        SW.add(i)

    with open("stopwords-ko.txt") as f:
        for word in f:
            SW.add(word)

    cleaned_text = []
    for doc in documents:
        temp_doc = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힇 ]","",doc)    
        cleaned_text.append(temp_doc)


    mecab = Mecab()
    tokenized_text = []
    tokenizer="noun"
    if tokenizer == "noun":
        for n in notebook.tqdm(range(len(cleaned_text)), desc="Preprocessing"):
            token_text = mecab.nouns(cleaned_text[n])
            token_text = [word for word in token_text if word not in SW and len(word) > 1]
            tokenized_text.append(token_text)
    elif tokenized == "morph":
        for n in notebook.tqdm(range(len(cleaned_text)), desc="Preprocessing"):
            token_text = mecab.morphs(cleaned_text[n])
            token_text = [word for word in token_text if word not in SW and len(word) > 1]
            tokenized_text.append(token_text)
    elif tokenized == "word":
        for n in notebook.tqdm(range(len(cleaned_text)), desc="Preprocessing"):
            token_text = cleaned_text[n].split()
            token_text = [word for word in token_text if word not in SW and len(word) > 1]
            tokenized_text.append(token_text)

    dictionary = corpora.Dictionary(tokenized_text)

    corpus = [dictionary.doc2bow(text) for text in tokenized_text]

    model = models.ldamodel.LdaModel(corpus, num_topics=link_count, id2word=dictionary)

    #model.show_topic(0, 10)

In [162]:
def get_topics():
    NUM_TOPICS = link_count 
    NUM_TOPIC_WORDS = 30 

    def build_doc_term_mat(documents):
        dictionary = corpora.Dictionary(documents)
        corpus = [dictionary.doc2bow(document) for document in documents]
        return corpus, dictionary


    documents=[]
    #for filename in glob.glob("content_data/*"):
        #documents += sc.textFile(filename).collect()
    
    documents += sc.textFile(glob.glob("content_data/*")[(len(glob.glob("content_data/*"))-1)]).collect()    
    #documents = read_documents(content_file_name)
    SW = define_stopwords("stopwords-ko.txt")
    cleaned_text = text_cleaning(documents)
    tokenized_text = text_tokenizing(cleaned_text, tokenizer="noun")

    dictionary = corpora.Dictionary(tokenized_text)

    corpus = [dictionary.doc2bow(text) for text in tokenized_text]

    model = models.ldamodel.LdaModel(corpus, num_topics=link_count, id2word=dictionary)

    # document-term matrix
    dictionary = corpora.Dictionary(tokenized_text)
    corpus = [dictionary.doc2bow(document) for document in tokenized_text]

    # LDA 실행
    model = models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha="auto", eta="auto")

    import pyLDAvis
    import pyLDAvis.gensim

    pyLDAvis.enable_notebook()

    data = pyLDAvis.gensim.prepare(model, corpus, dictionary)
    filecount=len(glob.glob("html_results/*"))
    pyLDAvis.save_html(data, 'html_results/lda_'+str(filecount)+'.html')
    #data

In [163]:
def save_topics():
    temp = data.topic_info.head(6)['Term'].values
    topic_words =""
    for i,v in enumerate(temp):
        if i == len(temp)-1:
            topic_words += v 
        else:
            topic_words += v + " & "
    print("end :: " + topic_words)

    filecount=len(glob.glob("keyword/keyword*"))
    sc.parallelize(topic_words).saveAsTextFile("keyword/keyword_"+str(filecount))